# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())                                       # checking your current working directory

filepath = os.getcwd() + '/event_data'                   # Get your current folder and subfolder event data
for root, dirs, files in os.walk(filepath):              # Create a for loop to create a list of files and collect each filepath
    file_path_list = glob.glob(os.path.join(root,'*'))   # join the file path and roots with the subdirectories using glob
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = []                                                                            # initiating an empty list of rows that will be generated from each file
    

for f in file_path_list:                                                                            # for every filepath in the file path list 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:                                    # reading csv file 
        csvreader = csv.reader(csvfile)                                                             # creating a csv reader object 
        next(csvreader)
        
        for line in csvreader:                                                                      # extracting each data row one by one and append it        
            full_data_rows_list.append(line) 
                                                                                                    
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:                                   # check the number of rows in your csv file
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()             # To establish connection and begin executing queries, need a session
print(session)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    keyspace = session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkdb
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
    )
    print(keyspace)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkdb')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### QUERY 1
#### Question: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
##### PRIMARY KEY COMPOSITE: sessionId, itemInSession
##### Reason: Since we are looking for activity within a specific sessionId and itemInSession. These 2 columns are unique enough to not have duplicates once queried.

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_item_history"
query = query + "(sessionId int, itemInSession int, artist varchar, song varchar, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)                                                                                  # skip header
    for line in csvreader:
        query1 = "INSERT INTO session_item_history (sessionId, itemInSession, artist, song, length)" # Assign the INSERT statements into the `query1` variable
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s)"                                             
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))      # Assign which column element should be assigned for each column in the INSERT statement

#### Query1 Description
##### The following query will output an artist, song, and length based on a specific sessionId and itemInSession event.

In [30]:
query1 = "select artist, song, length from session_item_history where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query1)                                                                   # query1 is executed and results are stored in rows variable
except Exception as e:
    print(e)
    
output = []                                                                                          # Output list variable is created to eventually hold query1 data
for row in rows:
    output.append(row)                                                                               # Output list is appended with data from each row

df = pd.DataFrame(output)                                                                            # pandas DataFrame is created out of Output list
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### QUERY 2
#### Question: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
##### PRIMARY KEY COMPOSITE: userId, sessionId 
##### CLUSTERING COLUMNS: song, itemInSession 
###### WITH CLUSTERING ORDER BY song ASC, itemInSession ASC
#### Reason: Since we are looking for a specific userId and sessionId, we will start by filtering for this specific PK Composite. In order to make the Primary Key unique we need to add clustering columns. Adding song adds uniqueness to the PK and by adding the itemInSession we are able to narrow it down to specific events.

In [31]:
query2 = "CREATE TABLE IF NOT EXISTS artist_song_history"
query2 = query2 + "(sessionId int, userId int, artist varchar, song varchar, itemInSession int, firstName varchar, lastName varchar, PRIMARY KEY((userId, sessionId), song, itemInSession)) WITH CLUSTERING ORDER BY (song ASC, itemInSession ASC)"
try:
    session.execute(query2)
except Exception as e:
    print(e)

In [32]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)                                                                                                      # skip header
    for line in csvreader:
        query2 = "INSERT INTO artist_song_history (sessionId, userId, artist, song, itemInSession, firstName, lastName)" # Assign the INSERT statements into the `query2` variable
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[8]), int(line[10]), line[0], line[9], int(line[3]), line[1], line[4]))         # Assign which column element should be assigned for each column in the INSERT statement

#### Query2 Description
##### The following query selects the artist, song, and user (firstName, lastName). Query is being filtered by a specific userId and sessionId. By including "with clustering order" in the table creation the songs in our query will automatically be ordered in ASC order.

In [33]:
query2 = """SELECT artist, song, firstName, lastName
              FROM artist_song_history
              WHERE userId = 10 AND sessionId = 182
             """
try:
    rows = session.execute(query2)                                                                                       # query2 is executed and results are stored in rows variable
except Exception as e:
    print(e)

output = []                                                                                                              # Output list variable is created to eventually hold query2 data
for row in rows:
    output.append(row)                                                                                                   # Output list is appended with data from each row

df = pd.DataFrame(output)                                                                                                # pandas DataFrame is created out of Output list
df 

,artist,song,firstname,lastname
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
3,Sebastien Tellier,Kilometer,Sylvie,Cruz


#### QUERY 3
#### Question: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
##### PRIMARY KEY COMPOSITE: song, userId
##### Reason: We are able to create a unique Primary Key from song and userId since t

In [34]:
query3 = "CREATE TABLE IF NOT EXISTS user_song_history"
query3 = query3 + "(userId int, sessionId int, itemInSession int, firstName varchar, lastName varchar, song varchar, PRIMARY KEY(song, userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)                                                                                             # skip header
    for line in csvreader:
        query3 = "INSERT INTO user_song_history (userId, sessionId, itemInSession, firstName, lastName, song)"  # Assign the INSERT statements into the `query3` variable
        query3 = query3 + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query3, (int(line[10]), int(line[8]), int(line[3]), line[1], line[4], line[9] ))        # Assign which column element should be assigned for each column in the INSERT statement

#### Query3 Description
##### The following query will ouput all users who have listened to the song 'All Hands Against His Own'. Since the PK is comprised of the userId and song, we can get the unique users for the requested song.

In [36]:
query3 = """SELECT firstName, lastName, song
              FROM user_song_history
              WHERE song = 'All Hands Against His Own'
             """
try:
    rows = session.execute(query3)                                                                              # query3 is executed and results are stored in rows variable
except Exception as e:
    print(e)
output = []                                                                                                     # Output list variable is created to eventually hold query3 data

for row in rows:
    output.append(row)                                                                                          # Output list is appended with data from each row

    df = pd.DataFrame(output)                                                                                   # pandas DataFrame is created out of Output list
df

,firstname,lastname,song
0,Jacqueline,Lynch,All Hands Against His Own
1,Tegan,Levine,All Hands Against His Own
2,Sara,Johnson,All Hands Against His Own


### Drop the tables before closing out the sessions

In [16]:
query1 = "DROP TABLE session_item_history"
query2 = "DROP TABLE artist_song_history"
query3 = "DROP TABLE user_song_history"
session.execute(query1)
session.execute(query2)
session.execute(query3)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()